# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 12 2022 9:01AM,241624,22,620547,"NBTY Global, Inc.",Executing
1,Jul 12 2022 9:01AM,241623,22,620546,"NBTY Global, Inc.",Released
2,Jul 12 2022 8:42AM,241621,19,ELI07132022.1,Eli Lilly and Company,Released
3,Jul 12 2022 8:37AM,241620,10,PRF-32704,Bio-PRF,Released
4,Jul 12 2022 8:30AM,241596,10,0085856911,ISDIN Corporation,Released
5,Jul 12 2022 8:30AM,241596,10,0085856339,ISDIN Corporation,Released
6,Jul 12 2022 8:30AM,241596,10,0085856350,ISDIN Corporation,Released
7,Jul 12 2022 8:30AM,241596,10,0085856349,ISDIN Corporation,Released
8,Jul 12 2022 8:30AM,241596,10,0085856487,ISDIN Corporation,Released
9,Jul 12 2022 8:30AM,241597,10,Enova-10274,Emerginnova,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
41,241619,Released,5
42,241620,Released,1
43,241621,Released,1
44,241623,Released,1
45,241624,Executing,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
241619,NaN,NaN,5.0
241620,NaN,NaN,1.0
241621,NaN,NaN,1.0
241623,NaN,NaN,1.0
241624,NaN,1.0,NaN


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241485,0.0,4.0,46.0
241486,0.0,0.0,50.0
241490,0.0,0.0,50.0
241502,0.0,0.0,42.0
241507,0.0,0.0,39.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241485,0,4,46
241486,0,0,50
241490,0,0,50
241502,0,0,42
241507,0,0,39


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241485,0,4,46
1,241486,0,0,50
2,241490,0,0,50
3,241502,0,0,42
4,241507,0,0,39


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241485,,4,46
1,241486,,,50
2,241490,,,50
3,241502,,,42
4,241507,,,39


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 12 2022 9:01AM,241624,22,"NBTY Global, Inc."
1,Jul 12 2022 9:01AM,241623,22,"NBTY Global, Inc."
2,Jul 12 2022 8:42AM,241621,19,Eli Lilly and Company
3,Jul 12 2022 8:37AM,241620,10,Bio-PRF
4,Jul 12 2022 8:30AM,241596,10,ISDIN Corporation
9,Jul 12 2022 8:30AM,241597,10,Emerginnova
10,Jul 12 2022 8:30AM,241598,10,"Methapharm, Inc."
17,Jul 12 2022 8:30AM,241587,10,ISDIN Corporation
36,Jul 12 2022 8:30AM,241584,10,ISDIN Corporation
78,Jul 12 2022 8:30AM,241594,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 12 2022 9:01AM,241624,22,"NBTY Global, Inc.",,1,
1,Jul 12 2022 9:01AM,241623,22,"NBTY Global, Inc.",,,1
2,Jul 12 2022 8:42AM,241621,19,Eli Lilly and Company,,,1
3,Jul 12 2022 8:37AM,241620,10,Bio-PRF,,,1
4,Jul 12 2022 8:30AM,241596,10,ISDIN Corporation,,,20
5,Jul 12 2022 8:30AM,241597,10,Emerginnova,,,1
6,Jul 12 2022 8:30AM,241598,10,"Methapharm, Inc.",,,7
7,Jul 12 2022 8:30AM,241587,10,ISDIN Corporation,,,59
8,Jul 12 2022 8:30AM,241584,10,ISDIN Corporation,,14,16
9,Jul 12 2022 8:30AM,241594,10,ISDIN Corporation,,,66


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 12 2022 9:01AM,241624,22,"NBTY Global, Inc.",,1,
1,Jul 12 2022 9:01AM,241623,22,"NBTY Global, Inc.",1,,
2,Jul 12 2022 8:42AM,241621,19,Eli Lilly and Company,1,,
3,Jul 12 2022 8:37AM,241620,10,Bio-PRF,1,,
4,Jul 12 2022 8:30AM,241596,10,ISDIN Corporation,20,,
5,Jul 12 2022 8:30AM,241597,10,Emerginnova,1,,
6,Jul 12 2022 8:30AM,241598,10,"Methapharm, Inc.",7,,
7,Jul 12 2022 8:30AM,241587,10,ISDIN Corporation,59,,
8,Jul 12 2022 8:30AM,241584,10,ISDIN Corporation,16,14,
9,Jul 12 2022 8:30AM,241594,10,ISDIN Corporation,66,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 12 2022 9:01AM,241624,22,"NBTY Global, Inc.",,1,
1,Jul 12 2022 9:01AM,241623,22,"NBTY Global, Inc.",1,,
2,Jul 12 2022 8:42AM,241621,19,Eli Lilly and Company,1,,
3,Jul 12 2022 8:37AM,241620,10,Bio-PRF,1,,
4,Jul 12 2022 8:30AM,241596,10,ISDIN Corporation,20,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 12 2022 9:01AM,241624,22,"NBTY Global, Inc.",NaN,1.0,NaN
1,Jul 12 2022 9:01AM,241623,22,"NBTY Global, Inc.",1.0,NaN,NaN
2,Jul 12 2022 8:42AM,241621,19,Eli Lilly and Company,1.0,NaN,NaN
3,Jul 12 2022 8:37AM,241620,10,Bio-PRF,1.0,NaN,NaN
4,Jul 12 2022 8:30AM,241596,10,ISDIN Corporation,20.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 12 2022 9:01AM,241624,22,"NBTY Global, Inc.",0.0,1.0,0.0
1,Jul 12 2022 9:01AM,241623,22,"NBTY Global, Inc.",1.0,0.0,0.0
2,Jul 12 2022 8:42AM,241621,19,Eli Lilly and Company,1.0,0.0,0.0
3,Jul 12 2022 8:37AM,241620,10,Bio-PRF,1.0,0.0,0.0
4,Jul 12 2022 8:30AM,241596,10,ISDIN Corporation,20.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,6038850,837.0,29.0,0.0
16,483118,2.0,0.0,0.0
19,1449549,9.0,7.0,17.0
20,483122,29.0,13.0,0.0
21,483225,2.0,0.0,0.0
22,483247,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,6038850,837.0,29.0,0.0
1,16,483118,2.0,0.0,0.0
2,19,1449549,9.0,7.0,17.0
3,20,483122,29.0,13.0,0.0
4,21,483225,2.0,0.0,0.0
5,22,483247,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,837.0,29.0,0.0
1,16,2.0,0.0,0.0
2,19,9.0,7.0,17.0
3,20,29.0,13.0,0.0
4,21,2.0,0.0,0.0
5,22,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,837.0
1,16,Released,2.0
2,19,Released,9.0
3,20,Released,29.0
4,21,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,20,21,22
Status,,,,,,
Completed,0.0,0.0,17.0,0.0,0.0,0.0
Executing,29.0,0.0,7.0,13.0,0.0,1.0
Released,837.0,2.0,9.0,29.0,2.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,20,21,22
0,Completed,0.0,0.0,17.0,0.0,0.0,0.0
1,Executing,29.0,0.0,7.0,13.0,0.0,1.0
2,Released,837.0,2.0,9.0,29.0,2.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,20,21,22
0,Completed,0.0,0.0,17.0,0.0,0.0,0.0
1,Executing,29.0,0.0,7.0,13.0,0.0,1.0
2,Released,837.0,2.0,9.0,29.0,2.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()